In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e8/sample_submission.csv
/kaggle/input/playground-series-s5e8/train.csv
/kaggle/input/playground-series-s5e8/test.csv


In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier

In [3]:
train = pd.read_csv("/kaggle/input/playground-series-s5e8/train.csv")
test = pd.read_csv("/kaggle/input/playground-series-s5e8/test.csv")
sample_submission = pd.read_csv("/kaggle/input/playground-series-s5e8/sample_submission.csv")

In [4]:
X = train.drop(["id","y"],axis = 1)
y = train["y"]

In [5]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
LEcols = ["month"]
OHcols = ["job","marital","education","default","housing","loan","contact","poutcome"]
num_cols = ["age", "balance", "duration", "campaign","day","pdays", "previous"]
# Define preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ("ohe", OneHotEncoder(handle_unknown="ignore"), OHcols),
        ("le", OrdinalEncoder(), LEcols),
        ("num", StandardScaler(), num_cols)
    ]
)

# Build pipeline
pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
   ("classifier", XGBClassifier(
        n_estimators=300,       # number of trees
        learning_rate=0.1,      # step size shrinkage
        max_depth=5,            # tree depth
        subsample=0.8,          # row sampling
        colsample_bytree=0.8,   # feature sampling
        random_state=42,
        use_label_encoder=False,
        eval_metric="logloss"   # avoids warning
    ))
])

In [7]:
pipeline.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('ohe',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['job', 'marital',
                                                   'education', 'default',
                                                   'housing', 'loan', 'contact',
                                                   'poutcome']),
                                                 ('le', OrdinalEncoder(),
                                                  ['month']),
                                                 ('num', StandardScaler(),
                                                  ['age', 'balance', 'duration',
                                                   'campaign', 'day', 'pdays',
                                                   'previous'])])),
                ('classifier',
                 XGBClassifier(base_s...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.1,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=5, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=300, n_jobs=None,
                               num_parallel_tree=None, random_state=42, ...))])

In [8]:
predictions = pipeline.predict_proba(test)[:,1]

In [9]:
submission = pd.DataFrame({"id":test["id"],
                          "y": predictions}) 
submission.to_csv("SUBDT001.csv",index = False)

In [10]:

accuracy = accuracy_score(pipeline.predict(X_val), y_val)
print("Accuracy:", accuracy)


Accuracy: 0.9337333333333333
